# Analysing the class distribution across tissues

In [74]:
import os
import pickle
import torch
import pandas as pd
from itertools import compress

from utils.knowledge_db import TISSUES

In [43]:
base_path = "/export/share/krausef99dm/data/"

cod_train_data_file = os.path.join(base_path, "data_train", "codon_train_8.1k_data.pkl")
cod_val_data_file = os.path.join(base_path, "data_test", "codon_val_8.1k_data.pkl")
cod_test_data_file = os.path.join(base_path, "data_test", "codon_test_8.1k_data.pkl")

nuc_train_data_file = os.path.join(base_path, "data_train", "train_9.0k_data.pkl")
nuc_val_data_file = os.path.join(base_path, "data_test", "val_9.0k_data.pkl")
nuc_test_data_file = os.path.join(base_path, "data_test", "test_9.0k_data.pkl")

In [79]:
def get_tissue_distrib(path):
    with open(path, 'rb') as f:
        rna_data, tissue_ids, targets, targets_bin =  pickle.load(f)
        
    # filter data
    mask = torch.ones((len(rna_data)), dtype=torch.bool)
    mask_bin = targets_bin > 0  # only keep low-/high-PTR samples
    mask = mask_bin & mask
    targets_bin -= 1  # make binary class 0/1 encoded

    if "cod" in path:
        max_seq_len = 2700
    else:
        max_seq_len = 9000
    
    mask_len = torch.tensor([len(d) <= max_seq_len for d in rna_data])
    mask = mask_len & mask
    
    rna_data = list(compress(rna_data, mask))
    tissue_ids = list(compress(tissue_ids.tolist(), mask))
    targets = list(compress(targets.tolist(), mask))
    targets_bin = list(compress(targets_bin.tolist(), mask))
    
    df = pd.DataFrame({"tissue_id": tissue_ids, "targets_bin": targets_bin,
                       "targets": targets})
    return df.groupby(['tissue_id', 'targets_bin']).targets.count()

In [80]:
# Get all distributions
cod_train_distrib, cod_val_distrib, cod_test_distrib, nuc_train_distrib, nuc_val_distrib, nuc_test_distrib = \
    [get_tissue_distrib(p) for p in [cod_train_data_file, cod_val_data_file, cod_test_data_file, nuc_train_data_file, nuc_val_data_file, nuc_test_data_file]]

In [90]:
cod_combined = pd.concat([cod_train_distrib, cod_val_distrib, cod_test_distrib], ignore_index=False, axis=1)
cod_combined.columns = ["train", "val", "test"]
cod_combined

train  val  test
tissue_id targets_bin                  
0         0              116   32    24
          1              149   44    23
1         0              418   86    89
          1              194   37    43
2         0              204   49    38
          1              523  124    81
3         0              207   54    44
          1              199   33    41
4         0              161   32    34
          1              144   30    25
5         0               92   24    24
          1              136   29    29
6         0              258   70    59
          1              158   29    31
7         0              139   37    27
          1              224   40    62
8         0              510   99   108
          1              194   29    38
9         0              177   36    44
          1              140   18    25
10        0              402   64    72
          1              253   59    51
11        0              274   71    66
          1              295   61    62
12        0              324   57    78
          1              206   51    40
13        0              303   63    55
          1              552  111   103
14        0              186   23    25
          1              177   46    35
15        0              369   82    63
          1              308   72    68
16        0              484  119    97
          1              346   73    71
17        0              194   32    30
          1              210   59    52
18        0              202   45    51
          1              196   46    31
19        0              114   32    25
          1              166   41    29
20        0              244   45    33
          1              218   50    45
21        0              113   21    29
          1              181   30    28
22        0              197   42    35
          1              223   50    60
23        0              133   42    37
          1              152   30    28
24        0              105   25    21
          1              135   41    34
25        0              233   37    59
          1              383   86    94
26        0              181   49    40
          1              363   73    73
27        0              220   61    43
          1              230   45    50
28        0              248   47    40
          1              133   38    31

In [ ]:
nuc_combined = pd.concat([nuc_train_distrib, nuc_val_distrib, nuc_test_distrib], ignore_index=False, axis=1)
nuc_combined.columns = ["train", "val", "test"]
nuc_combined.index = TISSUES
nuc_combined

In [108]:
inner_levels = cod_combined.index.droplevel(0)
new_outer = [t for t in TISSUES for _ in range(2)]

# Create new MultiIndexTISSUES
cod_combined.index = pd.MultiIndex.from_arrays(
    [new_outer, inner_levels.get_level_values(0)],
    names=['tissue', 'targets_bin']
)

nuc_combined.index = pd.MultiIndex.from_arrays(
    [new_outer, inner_levels.get_level_values(0)],
    names=['tissue', 'targets_bin']
)

In [118]:
combined = pd.concat([cod_combined, nuc_combined], ignore_index=False, axis=1)
combined.columns = [("Codon", "train"), ("Codon", "val"), ("Codon", "test"), 
                    ("Nucleotide", "train"), ("Nucleotide", "val"), ("Nucleotide", "test"), ]

combined.columns = pd.MultiIndex.from_tuples(combined.columns, names=['Encoding', 'Dataset'])
combined

Encoding                   Codon           Nucleotide          
Dataset                    train  val test      train  val test
tissue         targets_bin                                     
Adrenal        0             116   32   24        114   31   24
               1             149   44   23        144   41   23
Appendices     0             418   86   89        406   84   89
               1             194   37   43        181   36   43
Brain          0             204   49   38        196   47   38
               1             523  124   81        501  119   81
Colon          0             207   54   44        200   53   44
               1             199   33   41        191   31   41
Duodenum       0             161   32   34        157   32   34
               1             144   30   25        141   27   25
Uterus         0              92   24   24         87   24   24
               1             136   29   29        130   28   29
Esophagus      0             258   70   59        249   67   59
               1             158   29   31        153   29   31
Fallopiantube  0             139   37   27        136   35   27
               1             224   40   62        219   40   62
Fat            0             510   99  108        479   96  108
               1             194   29   38        186   27   38
Gallbladder    0             177   36   44        169   36   44
               1             140   18   25        132   18   25
Heart          0             402   64   72        391   64   72
               1             253   59   51        238   56   51
Kidney         0             274   71   66        259   66   66
               1             295   61   62        293   60   62
Liver          0             324   57   78        305   53   78
               1             206   51   40        194   50   40
Lung           0             303   63   55        292   59   55
               1             552  111  103        521  108  103
Lymphnode      0             186   23   25        176   23   25
               1             177   46   35        174   46   35
Ovary          0             369   82   63        354   78   63
               1             308   72   68        301   70   68
Pancreas       0             484  119   97        463  116   97
               1             346   73   71        333   70   71
Placenta       0             194   32   30        185   32   30
               1             210   59   52        196   55   52
Prostate       0             202   45   51        200   43   51
               1             196   46   31        181   43   31
Rectum         0             114   32   25        109   30   25
               1             166   41   29        163   38   29
Salivarygland  0             244   45   33        232   44   33
               1             218   50   45        209   47   45
Smallintestine 0             113   21   29        109   20   29
               1             181   30   28        174   28   28
Smoothmuscle   0             197   42   35        195   39   35
               1             223   50   60        210   49   60
Spleen         0             133   42   37        129   41   37
               1             152   30   28        151   28   28
Stomach        0             105   25   21        103   24   21
               1             135   41   34        126   40   34
Testis         0             233   37   59        223   36   59
               1             383   86   94        373   83   94
Thyroid        0             181   49   40        166   45   40
               1             363   73   73        353   72   73
Tonsil         0             220   61   43        214   58   43
               1             230   45   50        223   45   50
Urinarybladder 0             248   47   40        237   47   40
               1             133   38   31        129   36   31

In [124]:
# low PTR counts
combined[combined.index.get_level_values('targets_bin') == 0].sum()

Encoding    Dataset
Codon       train      6808
            val        1476
            test       1390
Nucleotide  train      6535
            val        1423
            test       1390
dtype: int64

In [125]:
# high PTR counts
combined[combined.index.get_level_values('targets_bin') == 1].sum()

Encoding    Dataset
Codon       train      6788
            val        1475
            test       1383
Nucleotide  train      6520
            val        1420
            test       1383
dtype: int64

In [121]:
# Total
combined.sum()

Encoding    Dataset
Codon       train      13596
            val         2951
            test        2773
Nucleotide  train      13055
            val         2843
            test        2773
dtype: int64